# Introducción a Machine Learning

***1. Explicar la diferencia entre un data-hub y un data-lake, y proponer un caso que ejemplifique el uso de cada uno.***

En este [link](https://blog.semarchy.com/how-to-differentiate-a-data-hub-a-data-lake-and-a-data-warehouse#:~:text=They%20all%20look%20similar%20but,on%20analytical%20uses%20of%20data.) se puede ver con más detalle la información, pero en esencia el data lake tiende a ser menos estructurado y a tener una menor calidad en el mismo, en comparación al data-hub.

***2. Un sitio de ventas de autos quiere agregar una nueva funcionalidad de forma que cuando un usuario
cargue su auto para la venta, el sitio le recomiende un precio para el mismo. Para ello quiere generar un modelo con XGBoost que al momento de cargar un nuevo aviso prediga el precio de venta.***

***El set de datos sobre las publicaciones de autos posee estos atributos:***

```
fecha, marca, modelo, versión, año, cantidad de puertas, segmento/tamaño, equipamiento, kilometraje, estado, transmisión, color, tipo de combustible, motor, potencia, único dueño, provincia, dueño directo, precio.
```

***Indicar el proceso de feature engineering que realizaría, indicando el detalle de los features que probaría para armar un modelo de predicción mediante XGBoost. Dar un ejemplo de un row del set de datos final.***


Primero de todo, sabemos que todos los features categóricos con los que vamos a terminar, deben ser encodeados a formato numérico.

Primero correría un random forest para ver que features actuan mejor, pues este modelo resulta ser sensible ante la diferencia de importancia entre columnas.

Me quedaría con las siguientes features:

```
fecha, segmento/tamaño, kilometraje, transmisión (columna binaria), potencia, único dueño (columna binaria), dueño directo (columna binaria)
```

Haría One Hot con:

```
marca, cantidad de puertas, estado, tipo de combustible, provincia 
```

Haría Mean Encoding con: 

```
modelo, versión, equipamiento, color, motor
```

Reemplazaría la columna `año` con `edad` (`2021 - año`).

Tenés que ordenar las fechas para evitar predecir cosas que ya pasaron.

***3. ¿Cuáles de las siguientes técnicas sirve para reducir el overfitting en XGBoost? Justificar.***

• ***Aumentar el número de estimadores;***

• ***Aumentar el valor del hiperparámetro gamma;***

• ***Reducir el valor del hiperparámetro min-child-weight;***

• ***Reducir el learning rate;***

• ***Reducir el valor del hiperparámetro max-depth.***

• *Aumentar el número de estimadores*: aumenta la cantidad de árboles lo que disminuye la probabilidad de tener decisiones sesgadas.

• *Aumentar el valor del hiperparámetro gamma*: [🤷‍♂️](https://stats.stackexchange.com/questions/418687/gamma-parameter-in-xgboost
)

• *Reducir el valor del hiperparámetro min-child-weight*: cappea el peso de cierto nodo, tal que evita que el mismo splittee tan fácil.

• *Reducir el learning rate*: 🤷‍♂️

• *Reducir el valor del hiperparámetro max-depth*: reduce el overfitting ya que, al tener menos hojas, evitamos tener predicciones tan rígidas y ajustadas a nuestro set de datos. 

***4. Dados los siguientes puntos en una dimensión y sus labels para un problema de clasificación binaria:***

```
(RED 0) (RED 0) (GREEN 1) (GREEN 0) (BLUE 0) (GREEN 0) (BLUE 0) (RED 1) (GREEN 1)
```

***aplicar dos algoritmos diferentes de mean encoding indicando cómo quedaría cada registro.***


Si aplicamos un `mean encoding` con $\frac{P}{P+N}$ tenemos:

```
(RED 1/3) (RED 1/3) (GREEN 1/2) (GREEN 1/2) (BLUE 0) (GREEN 1/2) (BLUE 0) (RED 1/3) (GREEN 1/2)
```

Y si aplicamos `mean encoding` con $\sum P$ tenemos:

```
(RED 1) (RED 1) (GREEN 2) (GREEN 2) (BLUE 0) (GREEN 2) (BLUE 0) (RED 1) (GREEN 2)
```

Siendo que $P$ y $N$ corresponden a los casos positivos y negativos respectivamente.

***5. Para el siguiente conjunto de registros, aplicar One-hot-encoding para la columna “¿Hijos?” y
Target-encoding para la columna “¿Trabajo?”. Sugerir además una forma de reducir posibles filtraciones de los labels para la columna codificada con Target-encoding.***


```
Reg    Sexo   ¿Hijos?   ¿Trabajo?   ¿Auto?   ¿Seguro de vida?
1      M       S         S           S        sí
2      F       N         N           N        no
3      F       N         S           N        no
4      M       N         S           S        no
5      M       S         S           S        si
6      M       S         N           S        sí
7      F       S         S           N        no
8      M       S         N           N        sí
9      F       N         N           N        no
10     F       N         S           S        sí
11     F       S         N           N        no
12     F       S         S           S        sí
```

Para hacer el target encoding sabemos que de las personas que trabajan, hay 4/7 con seguro de vida y de entre los vagos hay 2/5 con seguro de vida.

El resultado es:

```
Reg    Sexo    Con hijos    Sin hijos    ¿Trabajo?    ¿Auto?    ¿Seguro de vida?
1      M       1            0            4/7          S         sí
2      F       0            1            2/5          N         no
3      F       0            1            4/7          N         no
4      M       0            1            4/7          S         no
5      M       1            0            4/7          S         si
6      M       1            0            2/5          S         sí
7      F       1            0            4/7          N         no
8      M       1            0            2/5          N         sí
9      F       0            1            2/5          N         no
10     F       0            1            4/7          S         sí
11     F       1            0            2/5          N         no
12     F       1            0            4/7          S         sí
```

Podemos hacer *smoothing*. Primero calculamos `globalmean := mean(seguro de vida) = 6/12, alpha es un hiperparámetro, mean(target) := lo que está en trabajo ^^^` y en cada elemento dejamos:

``` 
(mean(target) * nrows + globalmean * alpha)/(nrows + alpha)
``` 

Recordemos que si $\alpha \rightarrow 0 \implies$ volvemos a tener nuestro dataset original y si $\alpha \rightarrow \infty$ todos los valores convergen en la media global.

Por ejemplo, en la primera fila tendríamos: $\frac{\frac{4}{7} \cdot 12 + \frac{6}{12} \cdot \alpha}{12 \cdot \alpha}$